# Image Classification with 2D CNN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
import zipfile
import os
import torch.nn.functional as F

In [ ]:
# Paths to the ZIP files
path_normal = '/content/drive/MyDrive/Normal.zip'
path_pneumonia = '/content/drive/MyDrive/Pneumonia.zip'
path_tuberculosis = '/content/drive/MyDrive/Tuberculosis.zip'

# Destination directories for extracted files
output_normal = '/content/drive/MyDrive/Extracted/Normal/'
output_pneumonia = '/content/drive/MyDrive/Extracted/Pneumonia/'
output_tuberculosis = '/content/drive/MyDrive/Extracted/Tuberculosis/'

# Function to extract ZIP files
def extract_zip(zip_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Create the directory if it doesn't exist
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Extracted {zip_path} to {output_dir}")

# Extract the contents of each ZIP file
extract_zip(path_normal, output_normal)
extract_zip(path_pneumonia, output_pneumonia)
extract_zip(path_tuberculosis, output_tuberculosis)


Extracted /content/drive/MyDrive/Normal.zip to /content/drive/MyDrive/Extracted/Normal/
Extracted /content/drive/MyDrive/Pneumonia.zip to /content/drive/MyDrive/Extracted/Pneumonia/
Extracted /content/drive/MyDrive/Tuberculosis.zip to /content/drive/MyDrive/Extracted/Tuberculosis/


In [ ]:
# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        img = img / 255.0  # Normalize pixel values to [0, 1]
        img = np.transpose(img, (2, 0, 1))  # Convert to (C, H, W) format for PyTorch
        if self.transform:
            img = self.transform(img)
        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Load images and labels
image_paths = []
labels = []

# Class-1 images (label 0)
for root, _, files in os.walk(output_normal):
    for file in files:
        image_paths.append(os.path.join(root, file))
        labels.append(0)

# Class-2 images (label 1)
for root, _, files in os.walk(output_pneumonia):
    for file in files:
        image_paths.append(os.path.join(root, file))
        labels.append(1)

# Class-3 images (label 2)
for root, _, files in os.walk(output_tuberculosis):
    for file in files:
        image_paths.append(os.path.join(root, file))
        labels.append(2)

# Convert to numpy arrays
labels = np.array(labels)

# Create Dataset and Split into Train/Validation/Test Sets
dataset = CustomDataset(image_paths=image_paths, labels=labels)
train_size = int(0.72 * len(dataset))  # 80% train-val split -> 90% training from train-val split
val_size = int(0.08 * len(dataset))   # 10% validation from train-val split
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# Define the Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=3)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.flatten = nn.Flatten()

        # CORRECTED LINEAR LAYER (calculate actual dimensions)
        self.fc1 = nn.Linear(32 * 13 * 13, 32)  # Changed from 4608 to 5408 equivalent
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)  # Removed softmax as CrossEntropyLoss includes it


model = CNNModel()

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Loop
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # Validation Loop
    model.eval()
    val_loss = 0.0
    correct_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels).item()

    val_accuracy = correct_predictions / len(val_dataset)
    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {val_accuracy:.4f}")

Epoch 1/50, Loss: 0.553723589900662
Validation Loss: 1.1773965656757355, Accuracy: 0.5833
Epoch 2/50, Loss: 0.17869891116724296
Validation Loss: 0.19228969886898994, Accuracy: 0.9417
Epoch 3/50, Loss: 0.13840104743619175
Validation Loss: 0.09877778496593237, Accuracy: 0.9750
Epoch 4/50, Loss: 0.08214390464127064
Validation Loss: 0.26993515715003014, Accuracy: 0.8750
Epoch 5/50, Loss: 0.06467841053381562
Validation Loss: 0.08230523765087128, Accuracy: 0.9833
Epoch 6/50, Loss: 0.03633807718699031
Validation Loss: 0.12514756433665752, Accuracy: 0.9500
Epoch 7/50, Loss: 0.04317768630297745
Validation Loss: 0.2200898751616478, Accuracy: 0.9167
Epoch 8/50, Loss: 0.031501638045644056
Validation Loss: 0.12414489779621363, Accuracy: 0.9583
Epoch 9/50, Loss: 0.02312765601912842
Validation Loss: 0.07742186170071363, Accuracy: 0.9667
Epoch 10/50, Loss: 0.01311324953156359
Validation Loss: 0.06479489430785179, Accuracy: 0.9750
Epoch 11/50, Loss: 0.008099660643732505
Validation Loss: 0.0880403863266

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

# Testing Loop with Full Metrics
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Store predictions and labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
kappa = cohen_kappa_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)



Test Accuracy: 0.9733
Precision: 0.9740
Recall: 0.9733
F1 Score: 0.9734
Cohen's Kappa: 0.9599

Confusion Matrix:
[[105   5   0]
 [  0  91   3]
 [  0   0  96]]


# 1D CNN for 12-lead ECG beat classification

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
X=np.load('/content/drive/MyDrive/x.npy')
y=np.load('/content/drive/MyDrive/y.npy')

In [ ]:
x_train_val, x_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.1, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))  # Number of unique classes in y
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

In [ ]:
# Print shapes for verification
print("X_train shape:", x_train.shape)
print("X_val shape:", x_val.shape)
print("X_test shape:", x_test.shape)
print("y_train_one_hot shape:", y_train_one_hot.shape)
print("y_val_one_hot shape:", y_val_one_hot.shape)
print("y_test_one_hot shape:", y_test_one_hot.shape)

X_train shape: torch.Size([2520, 12, 650])
X_val shape: torch.Size([280, 12, 650])
X_test shape: torch.Size([700, 12, 650])
y_train_one_hot shape: (2520, 7)
y_val_one_hot shape: (280, 7)
y_test_one_hot shape: (700, 7)


In [ ]:
# Convert data to PyTorch tensors with proper dimensions
# Original shape: (batch, 650, 12) → needs to become (batch, 12, 650) for PyTorch
x_train = torch.tensor(x_train.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays
x_val = torch.tensor(x_val.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays
x_test = torch.tensor(x_test.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays

# Convert one-hot encoded labels back to class indices
y_train = torch.tensor(y_train_one_hot.argmax(axis=1), dtype=torch.long) # Corrected variable name
y_val = torch.tensor(y_val_one_hot.argmax(axis=1), dtype=torch.long) # Corrected variable name
y_test = torch.tensor(y_test_one_hot.argmax(axis=1), dtype=torch.long) # Corrected variable name

# Create DataLoaders
batch_size = 32
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

<ipython-input-39-ffe42cb862ad>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train = torch.tensor(x_train.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays
<ipython-input-39-ffe42cb862ad>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val = torch.tensor(x_val.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays
<ipython-input-39-ffe42cb862ad>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test.swapaxes(1, 2), dtype=torch.float32)  # Use swapaxes for 3D arrays


In [ ]:
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        # Input: (batch_size, 650 channels, 12 timesteps)
        self.conv_block1 = nn.Sequential(
            nn.Conv1d(650, 16, kernel_size=5),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.AvgPool1d(kernel_size=2)  # Reduced from 10
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.AvgPool1d(kernel_size=2)  # Reduced from 5
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 2, 16),  # CORRECTED: 32×2=64 → 64→16
            nn.ReLU(),
            nn.Linear(16, 7)
        )

    def forward(self, x):
        x = self.conv_block1(x)  # Output: (batch, 16, 4)
        x = self.conv_block2(x)  # Output: (batch, 32, 2)
        return self.classifier(x)


# Initialize model, loss, and optimizer
model = CNN1D()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(100):  # Adjust number of epochs
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

        print(f"Epoch {epoch+1} | Val Loss: {val_loss/len(val_loader):.4f} | Acc: {correct/len(val_dataset):.4f}")


Epoch 1 | Val Loss: 0.0592 | Acc: 0.9893
Epoch 2 | Val Loss: 0.0753 | Acc: 0.9786
Epoch 3 | Val Loss: 0.0539 | Acc: 0.9929
Epoch 4 | Val Loss: 0.0608 | Acc: 0.9893
Epoch 5 | Val Loss: 0.0567 | Acc: 0.9893
Epoch 6 | Val Loss: 0.0621 | Acc: 0.9929
Epoch 7 | Val Loss: 0.0578 | Acc: 0.9857
Epoch 8 | Val Loss: 0.0540 | Acc: 0.9893
Epoch 9 | Val Loss: 0.0524 | Acc: 0.9893
Epoch 10 | Val Loss: 0.0627 | Acc: 0.9893
Epoch 11 | Val Loss: 0.0587 | Acc: 0.9893
Epoch 12 | Val Loss: 0.0669 | Acc: 0.9893
Epoch 13 | Val Loss: 0.1031 | Acc: 0.9679
Epoch 14 | Val Loss: 0.0705 | Acc: 0.9821
Epoch 15 | Val Loss: 0.0726 | Acc: 0.9821
Epoch 16 | Val Loss: 0.0523 | Acc: 0.9893
Epoch 17 | Val Loss: 0.0609 | Acc: 0.9893
Epoch 18 | Val Loss: 0.0565 | Acc: 0.9857
Epoch 19 | Val Loss: 0.0650 | Acc: 0.9893
Epoch 20 | Val Loss: 0.0659 | Acc: 0.9893
Epoch 21 | Val Loss: 0.0688 | Acc: 0.9893
Epoch 22 | Val Loss: 0.0658 | Acc: 0.9893
Epoch 23 | Val Loss: 0.0668 | Acc: 0.9893
Epoch 24 | Val Loss: 0.0742 | Acc: 0.9893
E

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

# Evaluation on test set
model.eval()
with torch.no_grad():
    test_loss = 0
    correct = 0
    all_preds = []
    all_labels = []

    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_loss += criterion(outputs, labels).item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

        # Store predictions and labels for metrics calculation
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = correct / len(test_dataset)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    kappa = cohen_kappa_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)

    # Print results
    print(f"Test Loss: {test_loss/len(test_loader):.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")
    print("\nConfusion Matrix:")
    print(conf_matrix)

    # Optional: Print classification report for more detailed metrics
    from sklearn.metrics import classification_report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, digits=4))


Test Loss: 0.0460
Accuracy: 0.9929
Precision: 0.9930
Recall: 0.9929
F1 Score: 0.9929
Cohen's Kappa: 0.9916

Confusion Matrix:
[[123   0   0   0   3   0   0]
 [  0 105   0   0   0   0   0]
 [  0   0  95   0   0   0   0]
 [  0   0   0 101   0   0   0]
 [  0   1   0   0  86   0   0]
 [  0   0   0   0   0  88   1]
 [  0   0   0   0   0   0  97]]

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.9762    0.9880       126
           1     0.9906    1.0000    0.9953       105
           2     1.0000    1.0000    1.0000        95
           3     1.0000    1.0000    1.0000       101
           4     0.9663    0.9885    0.9773        87
           5     1.0000    0.9888    0.9944        89
           6     0.9898    1.0000    0.9949        97

    accuracy                         0.9929       700
   macro avg     0.9924    0.9934    0.9928       700
weighted avg     0.9930    0.9929    0.9929       700

